In [8]:
import sys
import os
sys.path.append(os.path.abspath("../"))  # or "../../" depending on location

In [9]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
from utils.load_models import get_CNN, get_CNN_small, get_resnet, set_device

In [14]:

# --- 設定 ---
epsilon = 0.03
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_path = "../cifar10_fgsm_attack.pt"

# --- データ前処理（標準化なし） ---
transform_plain = transforms.ToTensor()


# ダウンロード済みのデータを指定して読み込み
root = "../data"
test_dataset = datasets.CIFAR10(
    root=root, 
    train=False, 
    download=False, 
    transform=transform_plain
)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# --- FGSM攻撃関数 ---
def fgsm_attack(model, images, labels, epsilon):
    images = images.clone().detach().to(device)
    labels = labels.to(device)
    images.requires_grad = True
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    model.zero_grad()
    loss.backward()
    grad_sign = images.grad.data.sign()
    adv_images = images + epsilon * grad_sign
    adv_images = torch.clamp(adv_images, 0, 1)
    return adv_images.detach()

# --- 敵対画像生成と保存 ---
def generate_and_save_adversarial_dataset(model,save_path,device = "cpu",):
    model.eval()
    model.to(device)

    orig_list = []
    adv_list = []
    label_list = []

    for images, labels in test_loader:
        adv_images = fgsm_attack(model, images, labels, epsilon)
        orig_list.append(images.cpu())
        adv_list.append(adv_images.cpu())
        label_list.append(labels.cpu())

    orig_tensor = torch.cat(orig_list)
    adv_tensor = torch.cat(adv_list)
    label_tensor = torch.cat(label_list)
    
    save_dir = os.path.dirname(save_path)
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
        
    torch.save({
        'original': orig_tensor,
        'adversarial': adv_tensor,
        'labels': label_tensor
    }, save_path)

    print(f"Saved adversarial dataset to: {save_path}")


In [15]:
device = set_device()
resnet = get_resnet().to(device)
save_path = "fgsm/cifar10_fgsm_attack_byResNet.pt"
generate_and_save_adversarial_dataset(resnet,save_path=save_path,device=device)


Saved adversarial dataset to: fgsm/cifar10_fgsm_attack_byResNet.pt


In [16]:
CNN = get_CNN().to(device)
save_path = "fgsm/cifar10_fgsm_attack_byCNN.pt"
generate_and_save_adversarial_dataset(CNN,save_path=save_path,device=device)

Saved adversarial dataset to: fgsm/cifar10_fgsm_attack_byCNN.pt


In [17]:
CNN_small = get_CNN_small().to(device)
save_path = "fgsm/cifar10_fgsm_attack_byCNN_small.pt"
generate_and_save_adversarial_dataset(CNN,save_path=save_path,device=device)

Saved adversarial dataset to: fgsm/cifar10_fgsm_attack_byCNN_small.pt
